In [8]:
from dotenv import load_dotenv
import os
load_dotenv()   
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
print(PINECONE_API_KEY)

pcsk_4aB64N_LzWfjp8q9AgJ2wM6zbV3v3nF8hxmCtKaKbXPbfDHYH4j9mg8doapR12BCjUMyx6


In [12]:
from pinecone.grpc import PineconeGRPC as Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)

# To get the unique host for an index, 
# see https://docs.pinecone.io/guides/data/target-an-index
index = pc.Index(host="synergyfinal")


In [17]:
from pinecone import Pinecone
import json
import uuid

pc = Pinecone(api_key=PINECONE_API_KEY)

# Load JSON file
with open("transcripts_with_timestamps.json", "r") as f:
    data = json.load(f)


embeddings = pc.inference.embed(
    model="llama-text-embed-v2",
    inputs=[d['text'] for d in data],
    parameters={
        "input_type": "passage"
    }
)

vectors = []
for d, e in zip(data, embeddings):
    vectors.append({
        "id": str(uuid.uuid4()),
        "values": e['values'],
        "metadata": {'text': d['text'], 'start': d['start'], 'end': d['end']}
    })


index=pc.Index('synergy')
print(index);
index.upsert(
    vectors=vectors,
    namespace="ns1"
)

{'upserted_count': 9}

In [ ]:
# Define your query
query = "what crystals are we talking about?"

# Convert the query into a numerical vector that Pinecone can search with
query_embedding = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[query],
    parameters={
        "input_type": "query"
    }
)



# Search the index for the three most similar vectors
results = index.query(
    namespace="ns1",
    vector=query_embedding[0].values,
    top_k=2,
    include_values=False,
    include_metadata=True
)


print(results)


{'matches': [{'id': '3fac198b-8c04-4c04-b94d-9033d8db6d4c',
              'metadata': {'end': 726.27,
                           'start': 606.27,
                           'text': 'on how much calcium citrate I recovered. '
                                   "For those of you who are interested I've "
                                   'included all of my calculations here, as '
                                   "well as some explanations. If you don't "
                                   "feel like reading it that's fine the only "
                                   'thing that you have to note is that I used '
                                   'slightly less of sulfuric acid than '
                                   'needed. I kept stirring it for several '
                                   'minutes to make sure that the reaction '
                                   'went to completion. Again went back to my '
                                   'trusty vacuum filter and I separat

In [26]:
results.matches[0].metadata['text']

"on how much calcium citrate I recovered. For those of you who are interested I've included all of my calculations here, as well as some explanations. If you don't feel like reading it that's fine the only thing that you have to note is that I used slightly less of sulfuric acid than needed. I kept stirring it for several minutes to make sure that the reaction went to completion. Again went back to my trusty vacuum filter and I separated off the calcium sulfate. Even with the vacuum on full the filtering process is kind of slow. Once everything comes through though I use a little bit of water to wash the beaker and then I added to the filter. Using a stir rod I mix things around to give the calcium sulfate a good wash. There might be some citric acid trapped in the calcium sulfate and this can help to pull it out. I repeated the washing step a couple times but it's important to use as little water as possible. little water as possible. All of the citric acid is dissolved in the water h

In [ ]:
relevant_text = ""
for i in results.matches:
    relevant_text += i.metadata['text'] + "---"



on how much calcium citrate I recovered. For those of you who are interested I've included all of my calculations here, as well as some explanations. If you don't feel like reading it that's fine the only thing that you have to note is that I used slightly less of sulfuric acid than needed. I kept stirring it for several minutes to make sure that the reaction went to completion. Again went back to my trusty vacuum filter and I separated off the calcium sulfate. Even with the vacuum on full the filtering process is kind of slow. Once everything comes through though I use a little bit of water to wash the beaker and then I added to the filter. Using a stir rod I mix things around to give the calcium sulfate a good wash. There might be some citric acid trapped in the calcium sulfate and this can help to pull it out. I repeated the washing step a couple times but it's important to use as little water as possible. little water as possible. All of the citric acid is dissolved in the water he

In [30]:
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
load_dotenv()
import os
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [37]:


def answer_using_relevant_text(relevant_text):
    
    prompt = """
    Your role is to answer the question based on the information provided.
    Be concise and to the point. Keep your answer short and simple using the information given yet comprehensive.
    You are given import information to answer the question. You are given relevant information to answer the question.
    Answer the question using the information provided.
    """


    llm = ChatGroq(temperature=0, groq_api_key=GROQ_API_KEY, model_name="llama-3.1-8b-instant")

    # Build the full prompt with the transcript text
    actual_prompt = ChatPromptTemplate.from_messages([
        ("system", prompt + relevant_text),
        ("human", "{input}")
    ])

    chain = actual_prompt | llm
    inputt = f"answer the question-{query} using the provided information"
    response = chain.invoke({"input": inputt})
    
    # Return the notes content
    return response.content

In [38]:
answer_using_relevant_text(relevant_text)

'We are talking about calcium citrate crystals.'